In [4]:
#!/usr/bin/env python
# coding: utf-8

# Import necessary libraries
import os
import datetime
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import cv2
import joblib
import warnings
import random
warnings.filterwarnings('ignore')

# Define paths
path = Path('csv_out')
eval_video_path = Path('eval_model_on_video')

# Create new output directory with timestamp to avoid overwriting
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
output_dir = eval_video_path / f'output_{timestamp}'
output_dir.mkdir(parents=True, exist_ok=True)
print(f'Outputs will be saved to: {output_dir}')

# Load data
df1 = pd.read_csv(path / 'tracking_data.csv')
df2 = pd.read_csv(path / 'overall_turn_label.csv')

# Merge dataframes
df_merged = pd.merge(df1, df2[['id', 'frame', 'overall_turn_label']], on=['id', 'frame'], how='left')

# Fill missing 'overall_turn_label' values by forward and backward filling per vehicle id
df_merged['overall_turn_label'] = df_merged.groupby('id')['overall_turn_label'].fillna(method='ffill')
df_merged['overall_turn_label'] = df_merged.groupby('id')['overall_turn_label'].fillna(method='bfill')

# Check for remaining missing values
missing_values = df_merged['overall_turn_label'].isnull().sum()
print(f"Number of missing 'overall_turn_label': {missing_values}")

# Fill any remaining missing values with 'straight'
df_merged['overall_turn_label'] = df_merged['overall_turn_label'].fillna('straight')

# One-Hot encode 'overall_turn_label'
encoder = OneHotEncoder(sparse_output=False)
turn_labels_encoded = encoder.fit_transform(df_merged[['overall_turn_label']])
turn_label_columns = encoder.get_feature_names_out(['overall_turn_label'])
df_merged[turn_label_columns] = turn_labels_encoded

# Define input features
input_features = ['center_x', 'center_y'] + list(turn_label_columns)

# Define sequence lengths
sequence_length = 90  # Input sequence length (90 frames, 3 seconds at 30 fps)
predict_length = 45   # Prediction sequence length (45 frames, 1.5 seconds at 30 fps)

# Generate input and target sequences for the current model (with turn feature)
input_sequences = []
target_sequences = []
sequence_vehicle_ids = []

grouped = df_merged.groupby('id')

for track_id, group in grouped:
    group = group.sort_values('frame').reset_index(drop=True)
    features = group[input_features].values

    num_sequences = len(features) - sequence_length - predict_length + 1
    if num_sequences <= 0:
        continue

    for i in range(num_sequences):
        input_seq = features[i:i + sequence_length]
        target_seq = features[i + sequence_length:i + sequence_length + predict_length, :2]  # Only 'center_x' and 'center_y'

        input_sequences.append(input_seq)
        target_sequences.append(target_seq)
        sequence_vehicle_ids.append(track_id)

# Convert to NumPy arrays
input_sequences = np.array(input_sequences)
target_sequences = np.array(target_sequences)
sequence_vehicle_ids = np.array(sequence_vehicle_ids)

# Data normalization
numeric_feature_indices = [0, 1]  # Indices for 'center_x' and 'center_y'

# Flatten the inputs for scaling
all_numeric_inputs = input_sequences[:, :, numeric_feature_indices].reshape(-1, len(numeric_feature_indices))

scaler = StandardScaler()
scaler.fit(all_numeric_inputs)

# Scale inputs
input_sequences[:, :, numeric_feature_indices] = scaler.transform(all_numeric_inputs).reshape(
    input_sequences.shape[0], input_sequences.shape[1], len(numeric_feature_indices))

# Scale targets
all_numeric_targets = target_sequences.reshape(-1, len(numeric_feature_indices))
target_sequences = scaler.transform(all_numeric_targets).reshape(
    target_sequences.shape[0], target_sequences.shape[1], len(numeric_feature_indices))

# Prepare baseline data (without turn feature)
input_features_baseline = ['center_x', 'center_y']

input_sequences_baseline = []
target_sequences_baseline = []
sequence_vehicle_ids_baseline = []

grouped = df_merged.groupby('id')

for track_id, group in grouped:
    group = group.sort_values('frame').reset_index(drop=True)
    features = group[input_features_baseline].values

    num_sequences = len(features) - sequence_length - predict_length + 1
    if num_sequences <= 0:
        continue

    for i in range(num_sequences):
        input_seq = features[i:i + sequence_length]
        target_seq = features[i + sequence_length:i + sequence_length + predict_length, :2]

        input_sequences_baseline.append(input_seq)
        target_sequences_baseline.append(target_seq)
        sequence_vehicle_ids_baseline.append(track_id)

# Convert to NumPy arrays
input_sequences_baseline = np.array(input_sequences_baseline)
target_sequences_baseline = np.array(target_sequences_baseline)
sequence_vehicle_ids_baseline = np.array(sequence_vehicle_ids_baseline)

# Data normalization for baseline
all_numeric_inputs_baseline = input_sequences_baseline.reshape(-1, len(numeric_feature_indices))

scaler_baseline = StandardScaler()
scaler_baseline.fit(all_numeric_inputs_baseline)

# Scale inputs
input_sequences_baseline = scaler_baseline.transform(all_numeric_inputs_baseline).reshape(
    input_sequences_baseline.shape[0], input_sequences_baseline.shape[1], len(numeric_feature_indices))

# Scale targets
all_numeric_targets_baseline = target_sequences_baseline.reshape(-1, len(numeric_feature_indices))
target_sequences_baseline = scaler_baseline.transform(all_numeric_targets_baseline).reshape(
    target_sequences_baseline.shape[0], target_sequences_baseline.shape[1], len(numeric_feature_indices))

# Define the model
class TrajectoryPredictor(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=2, output_size=2):
        super(TrajectoryPredictor, self).__init__()
        self.lstm_encoder = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.lstm_decoder = nn.LSTM(output_size, hidden_size, num_layers, batch_first=True)
        self.fc_out = nn.Linear(hidden_size, output_size)

    def forward(self, x, target_len):
        batch_size = x.size(0)

        # Encoder
        _, (hidden, cell) = self.lstm_encoder(x)

        # Decoder
        decoder_input = x[:, -1, :2].unsqueeze(1)  # Start with the last position of the input sequence
        outputs = []

        for t in range(target_len):
            out, (hidden, cell) = self.lstm_decoder(decoder_input, (hidden, cell))
            out = self.fc_out(out)
            outputs.append(out.squeeze(1))
            decoder_input = out  # Use current output as next input

        outputs = torch.stack(outputs, dim=1)
        return outputs

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Training parameters
batch_size = 1024
num_epochs = 10
target_len = predict_length  # Prediction length

# Prepare data for current model
input_size = input_sequences.shape[2]  # Number of input features
inputs = torch.tensor(input_sequences, dtype=torch.float32).to(device)
targets = torch.tensor(target_sequences, dtype=torch.float32).to(device)
model = TrajectoryPredictor(input_size=input_size, hidden_size=128, num_layers=2, output_size=2).to(device)

dataset = TensorDataset(inputs, targets)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

print(f'Total number of samples: {len(dataset)}')
print(f'Number of batches per epoch: {len(data_loader)}')

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop for current model
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch+1}/{num_epochs} for current model')
    model.train()
    total_loss = 0
    for batch_inputs, batch_targets in data_loader:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_inputs, target_len)

        # Compute loss
        loss = criterion(outputs, batch_targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(data_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}] for current model, Average Loss: {average_loss:.4f}')

# Prepare data for baseline model
input_size_baseline = input_sequences_baseline.shape[2]  # Number of input features (without turn feature)
inputs_baseline = torch.tensor(input_sequences_baseline, dtype=torch.float32).to(device)
targets_baseline = torch.tensor(target_sequences_baseline, dtype=torch.float32).to(device)
model_baseline = TrajectoryPredictor(input_size=input_size_baseline, hidden_size=128, num_layers=2, output_size=2).to(device)

dataset_baseline = TensorDataset(inputs_baseline, targets_baseline)
data_loader_baseline = DataLoader(dataset_baseline, batch_size=batch_size, shuffle=True)

print(f'Total number of samples for baseline: {len(dataset_baseline)}')
print(f'Number of batches per epoch for baseline: {len(data_loader_baseline)}')

# Define loss function and optimizer for baseline model
criterion_baseline = nn.MSELoss()
optimizer_baseline = torch.optim.Adam(model_baseline.parameters(), lr=0.001)

# Training loop for baseline model
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch+1}/{num_epochs} for baseline model')
    model_baseline.train()
    total_loss = 0
    for batch_inputs, batch_targets in data_loader_baseline:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)
        optimizer_baseline.zero_grad()

        # Forward pass
        outputs = model_baseline(batch_inputs, target_len)

        # Compute loss
        loss = criterion_baseline(outputs, batch_targets)

        # Backward pass and optimization
        loss.backward()
        optimizer_baseline.step()

        total_loss += loss.item()

    average_loss = total_loss / len(data_loader_baseline)
    print(f'Epoch [{epoch+1}/{num_epochs}] for baseline model, Average Loss: {average_loss:.4f}')

# Define function to compute metrics
def compute_metrics(predictions, targets, horizons):
    metrics = {}
    for horizon in horizons:
        outputs_at_horizon = predictions[:, :horizon, :]  # [num_samples, horizon, 2]
        targets_at_horizon = targets[:, :horizon, :]

        # Compute errors
        errors = outputs_at_horizon - targets_at_horizon  # [num_samples, horizon, 2]
        squared_errors = errors ** 2
        mse = squared_errors.mean().item()
        rmse = np.sqrt(mse)

        abs_errors = errors.abs()
        mae = abs_errors.mean().item()

        # Compute ADE
        displacement_errors = torch.norm(errors, dim=2)  # Euclidean distance over x and y
        ade = displacement_errors.mean().item()

        # Compute FDE
        final_errors = errors[:, -1, :]  # [num_samples, 2]
        fde = torch.norm(final_errors, dim=1).mean().item()

        metrics[horizon] = {
            'RMSE': rmse,
            'MAE': mae,
            'ADE': ade,
            'FDE': fde
        }

    return metrics

# Define prediction horizons
horizons = {
    15: 0.5,  # 0.5 seconds (15 frames)
    30: 1.0,  # 1.0 seconds (30 frames)
    45: 1.5   # 1.5 seconds (45 frames)
}

# Filter horizons not exceeding predict_length
horizons = {k: v for k, v in horizons.items() if k <= predict_length}

# Evaluate current model
model.eval()
with torch.no_grad():
    total_outputs = []
    total_targets = []
    for batch_inputs, batch_targets in data_loader:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)

        outputs = model(batch_inputs, target_len)
        total_outputs.append(outputs.cpu())
        total_targets.append(batch_targets.cpu())

    total_outputs = torch.cat(total_outputs, dim=0)
    total_targets = torch.cat(total_targets, dim=0)

    metrics = compute_metrics(total_outputs, total_targets, horizons.keys())

    # Save metrics to file
    metrics_file = output_dir / 'metrics_current_model.txt'
    with open(metrics_file, 'w') as f:
        for horizon_frames, time_sec in horizons.items():
            print(f'\nMetrics for current model at horizon: {time_sec} seconds ({horizon_frames} frames)')
            print(f"RMSE: {metrics[horizon_frames]['RMSE']:.4f}")
            print(f"MAE: {metrics[horizon_frames]['MAE']:.4f}")
            print(f"ADE: {metrics[horizon_frames]['ADE']:.4f}")
            print(f"FDE: {metrics[horizon_frames]['FDE']:.4f}")

            f.write(f'Metrics for current model at horizon: {time_sec} seconds ({horizon_frames} frames)\n')
            f.write(f"RMSE: {metrics[horizon_frames]['RMSE']:.4f}\n")
            f.write(f"MAE: {metrics[horizon_frames]['MAE']:.4f}\n")
            f.write(f"ADE: {metrics[horizon_frames]['ADE']:.4f}\n")
            f.write(f"FDE: {metrics[horizon_frames]['FDE']:.4f}\n\n")

# Evaluate baseline model
model_baseline.eval()
with torch.no_grad():
    total_outputs_baseline = []
    total_targets_baseline = []
    for batch_inputs, batch_targets in data_loader_baseline:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)

        outputs = model_baseline(batch_inputs, target_len)
        total_outputs_baseline.append(outputs.cpu())
        total_targets_baseline.append(batch_targets.cpu())

    total_outputs_baseline = torch.cat(total_outputs_baseline, dim=0)
    total_targets_baseline = torch.cat(total_targets_baseline, dim=0)

    metrics_baseline = compute_metrics(total_outputs_baseline, total_targets_baseline, horizons.keys())

    # Save metrics to file
    metrics_file_baseline = output_dir / 'metrics_baseline_model.txt'
    with open(metrics_file_baseline, 'w') as f:
        for horizon_frames, time_sec in horizons.items():
            print(f'\nMetrics for baseline model at horizon: {time_sec} seconds ({horizon_frames} frames)')
            print(f"RMSE: {metrics_baseline[horizon_frames]['RMSE']:.4f}")
            print(f"MAE: {metrics_baseline[horizon_frames]['MAE']:.4f}")
            print(f"ADE: {metrics_baseline[horizon_frames]['ADE']:.4f}")
            print(f"FDE: {metrics_baseline[horizon_frames]['FDE']:.4f}")

            # Corrected the mismatched quotation mark
            f.write(f"Metrics for baseline model at horizon: {time_sec} seconds ({horizon_frames} frames)\n")
            f.write(f"RMSE: {metrics_baseline[horizon_frames]['RMSE']:.4f}\n")
            f.write(f"MAE: {metrics_baseline[horizon_frames]['MAE']:.4f}\n")
            f.write(f"ADE: {metrics_baseline[horizon_frames]['ADE']:.4f}\n")
            f.write(f"FDE: {metrics_baseline[horizon_frames]['FDE']:.4f}\n\n")

# Save models and scalers
model_file = output_dir / f'trajectory_predictor_current_{timestamp}.pth'
scaler_file = output_dir / f'scaler_current_{timestamp}.save'

torch.save(model.state_dict(), model_file)
joblib.dump(scaler, scaler_file)
print(f'Current model saved to {model_file}')
print(f'Current scaler saved to {scaler_file}')

model_file_baseline = output_dir / f'trajectory_predictor_baseline_{timestamp}.pth'
scaler_file_baseline = output_dir / f'scaler_baseline_{timestamp}.save'

torch.save(model_baseline.state_dict(), model_file_baseline)
joblib.dump(scaler_baseline, scaler_file_baseline)
print(f'Baseline model saved to {model_file_baseline}')
print(f'Baseline scaler saved to {scaler_file_baseline}')

# Visualization (optional)
vehicle_ids_of_interest = [50, 328, 220, 46, 201, 238, 278, 185, 309, 303, 74, 93, 127, 203, 219, 210, 280, 390]

# Create vehicle ID to indices mapping for current model
vehicle_id_to_indices_current = {}
for vehicle_id in vehicle_ids_of_interest:
    indices = np.where(sequence_vehicle_ids == vehicle_id)[0]
    if len(indices) > 0:
        vehicle_id_to_indices_current[vehicle_id] = indices
    else:
        print(f"Vehicle ID {vehicle_id} not found in the current model sequences.")

# Create vehicle ID to indices mapping for baseline model
vehicle_id_to_indices_baseline = {}
for vehicle_id in vehicle_ids_of_interest:
    indices = np.where(sequence_vehicle_ids_baseline == vehicle_id)[0]
    if len(indices) > 0:
        vehicle_id_to_indices_baseline[vehicle_id] = indices
    else:
        print(f"Vehicle ID {vehicle_id} not found in the baseline model sequences.")

# Define visualization function
def visualize_prediction(model, inputs, targets, scaler, vehicle_id_to_indices, model_name):
    model.eval()
    with torch.no_grad():
        # Create index to vehicle ID mapping
        index_to_vehicle_id = {}
        for vehicle_id, indices in vehicle_id_to_indices.items():
            for idx in indices:
                index_to_vehicle_id[idx] = vehicle_id

        # Collect all indices
        all_indices = list(index_to_vehicle_id.keys())

        # Randomly select 5 indices
        num_samples = 5
        if len(all_indices) >= num_samples:
            selected_indices = random.sample(all_indices, num_samples)
        else:
            selected_indices = all_indices  # If less than 5 sequences are available

        for idx in selected_indices:
            test_input = inputs[idx].unsqueeze(0).to(device)
            true_target = targets[idx].to(device)

            # Perform prediction
            predicted_output = model(test_input, target_len)

            # Convert predictions and true targets to NumPy arrays
            predicted_output = predicted_output.squeeze(0).cpu().numpy()
            true_target = true_target.cpu().numpy()

            # Get historical input data for visualization
            history_input = test_input.squeeze(0).cpu().numpy()

            # Inverse scaling
            numeric_feature_indices = [0, 1]  # Indices of 'center_x' and 'center_y'

            # Inverse transform historical inputs
            history_input_numeric = history_input[:, numeric_feature_indices]
            history_input_unscaled = scaler.inverse_transform(history_input_numeric)

            # Inverse transform predicted outputs
            predicted_output_unscaled = scaler.inverse_transform(predicted_output)

            # Inverse transform true targets
            true_target_unscaled = scaler.inverse_transform(true_target)

            # Visualization
            plt.figure(figsize=(8, 6))

            # Plot historical trajectory
            plt.plot(history_input_unscaled[:, 0], history_input_unscaled[:, 1], 'bo-', label='Historical Trajectory')

            # Plot true future trajectory
            plt.plot(true_target_unscaled[:, 0], true_target_unscaled[:, 1], 'go-', label='True Future Trajectory')

            # Plot predicted future trajectory
            plt.plot(predicted_output_unscaled[:, 0], predicted_output_unscaled[:, 1], 'ro--', label='Predicted Future Trajectory')

            plt.legend()
            plt.xlabel('center_x')
            plt.ylabel('center_y')
            vehicle_id = index_to_vehicle_id.get(idx, 'Unknown')
            plt.title(f'{model_name} - Vehicle {vehicle_id} Trajectory Prediction (Seq {idx})')

            # Save figure
            figure_path = output_dir / f'{model_name}_vehicle_{vehicle_id}_seq_{idx}_{timestamp}.png'
            plt.savefig(figure_path)
            plt.close()
            print(f'Plot saved to {figure_path}')

# Visualize predictions for current model
visualize_prediction(model, inputs.cpu(), targets.cpu(), scaler, vehicle_id_to_indices_current, model_name='Current_Model')

# Visualize predictions for baseline model
visualize_prediction(model_baseline, inputs_baseline.cpu(), targets_baseline.cpu(), scaler_baseline, vehicle_id_to_indices_baseline, model_name='Baseline_Model')

# Combine metrics from both models into a DataFrame and save
metrics_combined = []

for horizon_frames, time_sec in horizons.items():
    metrics_combined.append({
        'Horizon (s)': time_sec,
        'Horizon (frames)': horizon_frames,
        'Model': 'Baseline',
        'RMSE': metrics_baseline[horizon_frames]['RMSE'],
        'MAE': metrics_baseline[horizon_frames]['MAE'],
        'ADE': metrics_baseline[horizon_frames]['ADE'],
        'FDE': metrics_baseline[horizon_frames]['FDE']
    })
    metrics_combined.append({
        'Horizon (s)': time_sec,
        'Horizon (frames)': horizon_frames,
        'Model': 'Current',
        'RMSE': metrics[horizon_frames]['RMSE'],
        'MAE': metrics[horizon_frames]['MAE'],
        'ADE': metrics[horizon_frames]['ADE'],
        'FDE': metrics[horizon_frames]['FDE']
    })

df_metrics = pd.DataFrame(metrics_combined)
metrics_csv_file = output_dir / 'metrics_comparison.csv'
df_metrics.to_csv(metrics_csv_file, index=False)
print(f'Metrics comparison saved to {metrics_csv_file}')

# Print comparison of performance metrics
print("\nComparison of Performance Metrics:")
print(df_metrics)


Outputs will be saved to: eval_model_on_video/output_20241018_074844
Number of missing 'overall_turn_label': 0
Using device: cuda
Total number of samples: 239560
Number of batches per epoch: 234
Starting epoch 1/10 for current model
Epoch [1/10] for current model, Average Loss: 0.1051
Starting epoch 2/10 for current model
Epoch [2/10] for current model, Average Loss: 0.0057
Starting epoch 3/10 for current model
Epoch [3/10] for current model, Average Loss: 0.0040
Starting epoch 4/10 for current model
Epoch [4/10] for current model, Average Loss: 0.0032
Starting epoch 5/10 for current model
Epoch [5/10] for current model, Average Loss: 0.0029
Starting epoch 6/10 for current model
Epoch [6/10] for current model, Average Loss: 0.0018
Starting epoch 7/10 for current model
Epoch [7/10] for current model, Average Loss: 0.0014
Starting epoch 8/10 for current model
Epoch [8/10] for current model, Average Loss: 0.0012
Starting epoch 9/10 for current model
Epoch [9/10] for current model, Average

In [5]:
# Import necessary libraries
import os
import datetime
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import cv2
import joblib
import warnings
import random
warnings.filterwarnings('ignore')

# Define paths
path = Path('csv_out')
eval_video_path = Path('eval_model_on_video')

# Create new output directory with timestamp to avoid overwriting
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
output_dir = eval_video_path / f'output_{timestamp}'
output_dir.mkdir(parents=True, exist_ok=True)
print(f'Outputs will be saved to: {output_dir}')

# Load data
df1 = pd.read_csv(path / 'tracking_data.csv')
df2 = pd.read_csv(path / 'overall_turn_label.csv')

# Merge dataframes
df_merged = pd.merge(df1, df2[['id', 'frame', 'overall_turn_label']], on=['id', 'frame'], how='left')

# Fill missing 'overall_turn_label' values by forward and backward filling per vehicle id
df_merged['overall_turn_label'] = df_merged.groupby('id')['overall_turn_label'].fillna(method='ffill')
df_merged['overall_turn_label'] = df_merged.groupby('id')['overall_turn_label'].fillna(method='bfill')

# Check for remaining missing values
missing_values = df_merged['overall_turn_label'].isnull().sum()
print(f"Number of missing 'overall_turn_label': {missing_values}")

# Fill any remaining missing values with 'straight' (if any)
df_merged['overall_turn_label'] = df_merged['overall_turn_label'].fillna('straight')

# One-Hot encode 'overall_turn_label'
encoder = OneHotEncoder(sparse_output=False)
turn_labels_encoded = encoder.fit_transform(df_merged[['overall_turn_label']])
turn_label_columns = encoder.get_feature_names_out(['overall_turn_label'])
df_merged[turn_label_columns] = turn_labels_encoded

# Define input features
input_features = ['center_x', 'center_y'] + list(turn_label_columns)

# Define sequence lengths
sequence_length = 90  # Input sequence length (90 frames, 3 seconds at 30 fps)
predict_length = 45   # Prediction sequence length (45 frames, 1.5 seconds at 30 fps)

# Generate input and target sequences for the current model (with turn feature)
input_sequences = []
target_sequences = []
sequence_vehicle_ids = []

grouped = df_merged.groupby('id')

for track_id, group in grouped:
    group = group.sort_values('frame').reset_index(drop=True)
    features = group[input_features].values

    num_sequences = len(features) - sequence_length - predict_length + 1
    if num_sequences <= 0:
        continue

    for i in range(num_sequences):
        input_seq = features[i:i + sequence_length]
        target_seq = features[i + sequence_length:i + sequence_length + predict_length, :2]  # Only 'center_x' and 'center_y'

        input_sequences.append(input_seq)
        target_sequences.append(target_seq)
        sequence_vehicle_ids.append(track_id)

# Convert to NumPy arrays
input_sequences = np.array(input_sequences)
target_sequences = np.array(target_sequences)
sequence_vehicle_ids = np.array(sequence_vehicle_ids)

# Data normalization
numeric_feature_indices = [0, 1]  # Indices for 'center_x' and 'center_y'

# Flatten the inputs for scaling
all_numeric_inputs = input_sequences[:, :, numeric_feature_indices].reshape(-1, len(numeric_feature_indices))

scaler = StandardScaler()
scaler.fit(all_numeric_inputs)

# Scale inputs
input_sequences[:, :, numeric_feature_indices] = scaler.transform(all_numeric_inputs).reshape(
    input_sequences.shape[0], input_sequences.shape[1], len(numeric_feature_indices))

# Scale targets
all_numeric_targets = target_sequences.reshape(-1, len(numeric_feature_indices))
target_sequences = scaler.transform(all_numeric_targets).reshape(
    target_sequences.shape[0], target_sequences.shape[1], len(numeric_feature_indices))

# Prepare baseline data (without turn feature)
input_features_baseline = ['center_x', 'center_y']

input_sequences_baseline = []
target_sequences_baseline = []
sequence_vehicle_ids_baseline = []

grouped = df_merged.groupby('id')

for track_id, group in grouped:
    group = group.sort_values('frame').reset_index(drop=True)
    features = group[input_features_baseline].values

    num_sequences = len(features) - sequence_length - predict_length + 1
    if num_sequences <= 0:
        continue

    for i in range(num_sequences):
        input_seq = features[i:i + sequence_length]
        target_seq = features[i + sequence_length:i + sequence_length + predict_length, :2]

        input_sequences_baseline.append(input_seq)
        target_sequences_baseline.append(target_seq)
        sequence_vehicle_ids_baseline.append(track_id)

# Convert to NumPy arrays
input_sequences_baseline = np.array(input_sequences_baseline)
target_sequences_baseline = np.array(target_sequences_baseline)
sequence_vehicle_ids_baseline = np.array(sequence_vehicle_ids_baseline)

# Data normalization for baseline
all_numeric_inputs_baseline = input_sequences_baseline.reshape(-1, len(numeric_feature_indices))

scaler_baseline = StandardScaler()
scaler_baseline.fit(all_numeric_inputs_baseline)

# Scale inputs
input_sequences_baseline = scaler_baseline.transform(all_numeric_inputs_baseline).reshape(
    input_sequences_baseline.shape[0], input_sequences_baseline.shape[1], len(numeric_feature_indices))

# Scale targets
all_numeric_targets_baseline = target_sequences_baseline.reshape(-1, len(numeric_feature_indices))
target_sequences_baseline = scaler_baseline.transform(all_numeric_targets_baseline).reshape(
    target_sequences_baseline.shape[0], target_sequences_baseline.shape[1], len(numeric_feature_indices))

# Define the model
class TrajectoryPredictor(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=2, output_size=2):
        super(TrajectoryPredictor, self).__init__()
        self.lstm_encoder = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.lstm_decoder = nn.LSTM(output_size, hidden_size, num_layers, batch_first=True)
        self.fc_out = nn.Linear(hidden_size, output_size)

    def forward(self, x, target_len):
        batch_size = x.size(0)

        # Encoder
        _, (hidden, cell) = self.lstm_encoder(x)

        # Decoder
        decoder_input = x[:, -1, :2].unsqueeze(1)  # Start with the last position of the input sequence
        outputs = []

        for t in range(target_len):
            out, (hidden, cell) = self.lstm_decoder(decoder_input, (hidden, cell))
            out = self.fc_out(out)
            outputs.append(out.squeeze(1))
            decoder_input = out  # Use current output as next input

        outputs = torch.stack(outputs, dim=1)
        return outputs

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Training parameters
batch_size = 1024
num_epochs = 10
target_len = predict_length  # Prediction length

# Prepare data for current model
input_size = input_sequences.shape[2]  # Number of input features
inputs = torch.tensor(input_sequences, dtype=torch.float32).to(device)
targets = torch.tensor(target_sequences, dtype=torch.float32).to(device)
model = TrajectoryPredictor(input_size=input_size, hidden_size=128, num_layers=2, output_size=2).to(device)

dataset = TensorDataset(inputs, targets)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

print(f'Total number of samples: {len(dataset)}')
print(f'Number of batches per epoch: {len(data_loader)}')

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop for current model
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch+1}/{num_epochs} for current model')
    model.train()
    total_loss = 0
    for batch_inputs, batch_targets in data_loader:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_inputs, target_len)

        # Compute loss
        loss = criterion(outputs, batch_targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(data_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}] for current model, Average Loss: {average_loss:.4f}')

# Prepare data for baseline model
input_size_baseline = input_sequences_baseline.shape[2]  # Number of input features (without turn feature)
inputs_baseline = torch.tensor(input_sequences_baseline, dtype=torch.float32).to(device)
targets_baseline = torch.tensor(target_sequences_baseline, dtype=torch.float32).to(device)
model_baseline = TrajectoryPredictor(input_size=input_size_baseline, hidden_size=128, num_layers=2, output_size=2).to(device)

dataset_baseline = TensorDataset(inputs_baseline, targets_baseline)
data_loader_baseline = DataLoader(dataset_baseline, batch_size=batch_size, shuffle=True)

print(f'Total number of samples for baseline: {len(dataset_baseline)}')
print(f'Number of batches per epoch for baseline: {len(data_loader_baseline)}')

# Define loss function and optimizer for baseline model
criterion_baseline = nn.MSELoss()
optimizer_baseline = torch.optim.Adam(model_baseline.parameters(), lr=0.001)

# Training loop for baseline model
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch+1}/{num_epochs} for baseline model')
    model_baseline.train()
    total_loss = 0
    for batch_inputs, batch_targets in data_loader_baseline:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)
        optimizer_baseline.zero_grad()

        # Forward pass
        outputs = model_baseline(batch_inputs, target_len)

        # Compute loss
        loss = criterion_baseline(outputs, batch_targets)

        # Backward pass and optimization
        loss.backward()
        optimizer_baseline.step()

        total_loss += loss.item()

    average_loss = total_loss / len(data_loader_baseline)
    print(f'Epoch [{epoch+1}/{num_epochs}] for baseline model, Average Loss: {average_loss:.4f}')

# Define function to compute metrics
def compute_metrics(predictions, targets, horizons):
    metrics = {}
    for horizon in horizons:
        outputs_at_horizon = predictions[:, :horizon, :]  # [num_samples, horizon, 2]
        targets_at_horizon = targets[:, :horizon, :]

        # Compute errors
        errors = outputs_at_horizon - targets_at_horizon  # [num_samples, horizon, 2]
        squared_errors = errors ** 2
        mse = squared_errors.mean().item()
        rmse = np.sqrt(mse)

        abs_errors = errors.abs()
        mae = abs_errors.mean().item()

        # Compute ADE
        displacement_errors = torch.norm(errors, dim=2)  # Euclidean distance over x and y
        ade = displacement_errors.mean().item()

        # Compute FDE
        final_errors = errors[:, -1, :]  # [num_samples, 2]
        fde = torch.norm(final_errors, dim=1).mean().item()

        metrics[horizon] = {
            'RMSE': rmse,
            'MAE': mae,
            'ADE': ade,
            'FDE': fde
        }

    return metrics

# Define prediction horizons
horizons = {
    15: 0.5,  # 0.5 seconds (15 frames)
    30: 1.0,  # 1.0 seconds (30 frames)
    45: 1.5   # 1.5 seconds (45 frames)
}

# Filter horizons not exceeding predict_length
horizons = {k: v for k, v in horizons.items() if k <= predict_length}

# Evaluate current model
model.eval()
with torch.no_grad():
    total_outputs = []
    total_targets = []
    for batch_inputs, batch_targets in data_loader:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)

        outputs = model(batch_inputs, target_len)
        total_outputs.append(outputs.cpu())
        total_targets.append(batch_targets.cpu())

    total_outputs = torch.cat(total_outputs, dim=0)
    total_targets = torch.cat(total_targets, dim=0)

    metrics = compute_metrics(total_outputs, total_targets, horizons.keys())

    # Save metrics to file
    metrics_file = output_dir / 'metrics_current_model.txt'
    with open(metrics_file, 'w') as f:
        for horizon_frames, time_sec in horizons.items():
            print(f'\nMetrics for current model at horizon: {time_sec} seconds ({horizon_frames} frames)')
            print(f"RMSE: {metrics[horizon_frames]['RMSE']:.4f}")
            print(f"MAE: {metrics[horizon_frames]['MAE']:.4f}")
            print(f"ADE: {metrics[horizon_frames]['ADE']:.4f}")
            print(f"FDE: {metrics[horizon_frames]['FDE']:.4f}")

            f.write(f'Metrics for current model at horizon: {time_sec} seconds ({horizon_frames} frames)\n')
            f.write(f"RMSE: {metrics[horizon_frames]['RMSE']:.4f}\n")
            f.write(f"MAE: {metrics[horizon_frames]['MAE']:.4f}\n")
            f.write(f"ADE: {metrics[horizon_frames]['ADE']:.4f}\n")
            f.write(f"FDE: {metrics[horizon_frames]['FDE']:.4f}\n\n")

# Evaluate baseline model
model_baseline.eval()
with torch.no_grad():
    total_outputs_baseline = []
    total_targets_baseline = []
    for batch_inputs, batch_targets in data_loader_baseline:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)

        outputs = model_baseline(batch_inputs, target_len)
        total_outputs_baseline.append(outputs.cpu())
        total_targets_baseline.append(batch_targets.cpu())

    total_outputs_baseline = torch.cat(total_outputs_baseline, dim=0)
    total_targets_baseline = torch.cat(total_targets_baseline, dim=0)

    metrics_baseline = compute_metrics(total_outputs_baseline, total_targets_baseline, horizons.keys())

    # Save metrics to file
    metrics_file_baseline = output_dir / 'metrics_baseline_model.txt'
    with open(metrics_file_baseline, 'w') as f:
        for horizon_frames, time_sec in horizons.items():
            print(f'\nMetrics for baseline model at horizon: {time_sec} seconds ({horizon_frames} frames)')
            print(f"RMSE: {metrics_baseline[horizon_frames]['RMSE']:.4f}")
            print(f"MAE: {metrics_baseline[horizon_frames]['MAE']:.4f}")
            print(f"ADE: {metrics_baseline[horizon_frames]['ADE']:.4f}")
            print(f"FDE: {metrics_baseline[horizon_frames]['FDE']:.4f}")

            f.write(f"Metrics for baseline model at horizon: {time_sec} seconds ({horizon_frames} frames)\n")
            f.write(f"RMSE: {metrics_baseline[horizon_frames]['RMSE']:.4f}\n")
            f.write(f"MAE: {metrics_baseline[horizon_frames]['MAE']:.4f}\n")
            f.write(f"ADE: {metrics_baseline[horizon_frames]['ADE']:.4f}\n")
            f.write(f"FDE: {metrics_baseline[horizon_frames]['FDE']:.4f}\n\n")

# Save models and scalers
model_file = output_dir / f'trajectory_predictor_current_{timestamp}.pth'
scaler_file = output_dir / f'scaler_current_{timestamp}.save'

torch.save(model.state_dict(), model_file)
joblib.dump(scaler, scaler_file)
print(f'Current model saved to {model_file}')
print(f'Current scaler saved to {scaler_file}')

model_file_baseline = output_dir / f'trajectory_predictor_baseline_{timestamp}.pth'
scaler_file_baseline = output_dir / f'scaler_baseline_{timestamp}.save'

torch.save(model_baseline.state_dict(), model_file_baseline)
joblib.dump(scaler_baseline, scaler_file_baseline)
print(f'Baseline model saved to {model_file_baseline}')
print(f'Baseline scaler saved to {scaler_file_baseline}')

# Visualization and metrics comparison code remains the same
# ...

# ---------------------------------
# Evaluate models on turning vehicles
# ---------------------------------

# 1. Filter data for turning vehicles based on 'overall_turn_label'
turn_labels = ['left_turn', 'right_turn']

# Since your 'overall_turn_label' represents the true turning status, use it directly
turning_df = df_merged[df_merged['overall_turn_label'].isin(turn_labels)]

# 2. Prepare sequences for turning vehicles for current model
input_sequences_turn = []
target_sequences_turn = []

grouped_turning = turning_df.groupby('id')

for track_id, group in grouped_turning:
    group = group.sort_values('frame').reset_index(drop=True)
    features = group[input_features].values

    num_sequences = len(features) - sequence_length - predict_length + 1
    if num_sequences <= 0:
        continue

    for i in range(num_sequences):
        input_seq = features[i:i + sequence_length]
        target_seq = features[i + sequence_length:i + sequence_length + predict_length, :2]  # Only 'center_x' and 'center_y'

        input_sequences_turn.append(input_seq)
        target_sequences_turn.append(target_seq)

# Convert to NumPy arrays
input_sequences_turn = np.array(input_sequences_turn)
target_sequences_turn = np.array(target_sequences_turn)

# Data normalization (using the same scaler)
numeric_feature_indices = [0, 1]  # 'center_x', 'center_y'

# Scale inputs
all_numeric_inputs_turn = input_sequences_turn[:, :, numeric_feature_indices].reshape(-1, len(numeric_feature_indices))
input_sequences_turn[:, :, numeric_feature_indices] = scaler.transform(all_numeric_inputs_turn).reshape(
    input_sequences_turn.shape[0], input_sequences_turn.shape[1], len(numeric_feature_indices))

# Scale targets
all_numeric_targets_turn = target_sequences_turn.reshape(-1, len(numeric_feature_indices))
target_sequences_turn = scaler.transform(all_numeric_targets_turn).reshape(
    target_sequences_turn.shape[0], target_sequences_turn.shape[1], len(numeric_feature_indices))

# Create dataset and dataloader
inputs_turn = torch.tensor(input_sequences_turn, dtype=torch.float32).to(device)
targets_turn = torch.tensor(target_sequences_turn, dtype=torch.float32).to(device)

dataset_turn = TensorDataset(inputs_turn, targets_turn)
data_loader_turn = DataLoader(dataset_turn, batch_size=batch_size, shuffle=False)

# 3. Evaluate current model on turning vehicles
def evaluate_model_on_turning(model, data_loader, target_len):
    model.eval()
    with torch.no_grad():
        total_outputs = []
        total_targets = []
        for batch_inputs, batch_targets in data_loader:
            batch_inputs = batch_inputs.to(device)
            batch_targets = batch_targets.to(device)

            outputs = model(batch_inputs, target_len)
            total_outputs.append(outputs.cpu())
            total_targets.append(batch_targets.cpu())

        total_outputs = torch.cat(total_outputs, dim=0)
        total_targets = torch.cat(total_targets, dim=0)

        metrics = compute_metrics(total_outputs, total_targets, horizons.keys())
    return metrics

metrics_turn_current = evaluate_model_on_turning(model, data_loader_turn, target_len)

# 4. Prepare data for baseline model on turning vehicles
# Remove turn features
input_sequences_turn_baseline = input_sequences_turn[:, :, :2]  # Only 'center_x' and 'center_y'

# Scale inputs using baseline scaler
all_numeric_inputs_turn_baseline = input_sequences_turn_baseline.reshape(-1, len(numeric_feature_indices))
input_sequences_turn_baseline = scaler_baseline.transform(all_numeric_inputs_turn_baseline).reshape(
    input_sequences_turn_baseline.shape[0], input_sequences_turn_baseline.shape[1], len(numeric_feature_indices))

# Scale targets using baseline scaler
all_numeric_targets_turn_baseline = target_sequences_turn.reshape(-1, len(numeric_feature_indices))
target_sequences_turn_baseline = scaler_baseline.transform(all_numeric_targets_turn_baseline).reshape(
    target_sequences_turn_baseline.shape[0], target_sequences_turn_baseline.shape[1], len(numeric_feature_indices))

# Create dataset and dataloader
inputs_turn_baseline = torch.tensor(input_sequences_turn_baseline, dtype=torch.float32).to(device)
targets_turn_baseline = torch.tensor(target_sequences_turn_baseline, dtype=torch.float32).to(device)

dataset_turn_baseline = TensorDataset(inputs_turn_baseline, targets_turn_baseline)
data_loader_turn_baseline = DataLoader(dataset_turn_baseline, batch_size=batch_size, shuffle=False)

# Evaluate baseline model on turning vehicles
metrics_turn_baseline = evaluate_model_on_turning(model_baseline, data_loader_turn_baseline, target_len)

# 5. Compare results
metrics_file_turn = output_dir / 'metrics_turning_vehicles.txt'
with open(metrics_file_turn, 'w') as f:
    for horizon_frames, time_sec in horizons.items():
        print(f'\nMetrics on turning vehicles at horizon: {time_sec} seconds ({horizon_frames} frames)')
        print(f"Baseline Model:")
        print(f"  RMSE: {metrics_turn_baseline[horizon_frames]['RMSE']:.4f}")
        print(f"  MAE: {metrics_turn_baseline[horizon_frames]['MAE']:.4f}")
        print(f"  ADE: {metrics_turn_baseline[horizon_frames]['ADE']:.4f}")
        print(f"  FDE: {metrics_turn_baseline[horizon_frames]['FDE']:.4f}")
        print(f"Current Model:")
        print(f"  RMSE: {metrics_turn_current[horizon_frames]['RMSE']:.4f}")
        print(f"  MAE: {metrics_turn_current[horizon_frames]['MAE']:.4f}")
        print(f"  ADE: {metrics_turn_current[horizon_frames]['ADE']:.4f}")
        print(f"  FDE: {metrics_turn_current[horizon_frames]['FDE']:.4f}")

        f.write(f'Metrics on turning vehicles at horizon: {time_sec} seconds ({horizon_frames} frames)\n')
        f.write(f"Baseline Model:\n")
        f.write(f"  RMSE: {metrics_turn_baseline[horizon_frames]['RMSE']:.4f}\n")
        f.write(f"  MAE: {metrics_turn_baseline[horizon_frames]['MAE']:.4f}\n")
        f.write(f"  ADE: {metrics_turn_baseline[horizon_frames]['ADE']:.4f}\n")
        f.write(f"  FDE: {metrics_turn_baseline[horizon_frames]['FDE']:.4f}\n")
        f.write(f"Current Model:\n")
        f.write(f"  RMSE: {metrics_turn_current[horizon_frames]['RMSE']:.4f}\n")
        f.write(f"  MAE: {metrics_turn_current[horizon_frames]['MAE']:.4f}\n")
        f.write(f"  ADE: {metrics_turn_current[horizon_frames]['ADE']:.4f}\n")
        f.write(f"  FDE: {metrics_turn_current[horizon_frames]['FDE']:.4f}\n\n")

print(f"Metrics on turning vehicles saved to {metrics_file_turn}")


Outputs will be saved to: eval_model_on_video/output_20241018_080535
Number of missing 'overall_turn_label': 0
Using device: cuda
Total number of samples: 239560
Number of batches per epoch: 234
Starting epoch 1/10 for current model
Epoch [1/10] for current model, Average Loss: 0.1196
Starting epoch 2/10 for current model
Epoch [2/10] for current model, Average Loss: 0.0049
Starting epoch 3/10 for current model
Epoch [3/10] for current model, Average Loss: 0.0034
Starting epoch 4/10 for current model
Epoch [4/10] for current model, Average Loss: 0.0026
Starting epoch 5/10 for current model
Epoch [5/10] for current model, Average Loss: 0.0021
Starting epoch 6/10 for current model
Epoch [6/10] for current model, Average Loss: 0.0018
Starting epoch 7/10 for current model
Epoch [7/10] for current model, Average Loss: 0.0015
Starting epoch 8/10 for current model
Epoch [8/10] for current model, Average Loss: 0.0013
Starting epoch 9/10 for current model
Epoch [9/10] for current model, Average

NameError: name 'target_sequences_turn_baseline' is not defined

In [6]:
# 导入必要的库
import os
import datetime
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import cv2
import joblib
import warnings
import random
warnings.filterwarnings('ignore')

# 定义路径
path = Path('csv_out')
eval_video_path = Path('eval_model_on_video')

# 创建带有时间戳的新输出目录，避免覆盖
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
output_dir = eval_video_path / f'output_{timestamp}'
output_dir.mkdir(parents=True, exist_ok=True)
print(f'Outputs will be saved to: {output_dir}')

# 加载数据
df1 = pd.read_csv(path / 'tracking_data.csv')
df2 = pd.read_csv(path / 'overall_turn_label.csv')

# 合并数据帧
df_merged = pd.merge(df1, df2[['id', 'frame', 'overall_turn_label']], on=['id', 'frame'], how='left')

# 按车辆ID填充缺失的 'overall_turn_label' 值（前向和后向填充）
df_merged['overall_turn_label'] = df_merged.groupby('id')['overall_turn_label'].fillna(method='ffill')
df_merged['overall_turn_label'] = df_merged.groupby('id')['overall_turn_label'].fillna(method='bfill')

# 检查剩余的缺失值
missing_values = df_merged['overall_turn_label'].isnull().sum()
print(f"Number of missing 'overall_turn_label': {missing_values}")

# 填充任何剩余的缺失值为 'straight'（如果有）
df_merged['overall_turn_label'] = df_merged['overall_turn_label'].fillna('straight')

# One-Hot 编码 'overall_turn_label'
encoder = OneHotEncoder(sparse_output=False)
turn_labels_encoded = encoder.fit_transform(df_merged[['overall_turn_label']])
turn_label_columns = encoder.get_feature_names_out(['overall_turn_label'])
df_merged[turn_label_columns] = turn_labels_encoded

# 定义输入特征
input_features = ['center_x', 'center_y'] + list(turn_label_columns)

# 定义序列长度
sequence_length = 90  # 输入序列长度（90帧，30fps下为3秒）
predict_length = 45   # 预测序列长度（45帧，30fps下为1.5秒）

# 生成当前模型的输入和目标序列（包含转弯特征）
input_sequences = []
target_sequences = []
sequence_vehicle_ids = []

grouped = df_merged.groupby('id')

for track_id, group in grouped:
    group = group.sort_values('frame').reset_index(drop=True)
    features = group[input_features].values

    num_sequences = len(features) - sequence_length - predict_length + 1
    if num_sequences <= 0:
        continue

    for i in range(num_sequences):
        input_seq = features[i:i + sequence_length]
        target_seq = features[i + sequence_length:i + sequence_length + predict_length, :2]  # 只取 'center_x' 和 'center_y'

        input_sequences.append(input_seq)
        target_sequences.append(target_seq)
        sequence_vehicle_ids.append(track_id)

# 转换为 NumPy 数组
input_sequences = np.array(input_sequences)
target_sequences = np.array(target_sequences)
sequence_vehicle_ids = np.array(sequence_vehicle_ids)

# 数据标准化
numeric_feature_indices = [0, 1]  # 'center_x' 和 'center_y' 的索引

# 扁平化输入以进行缩放
all_numeric_inputs = input_sequences[:, :, numeric_feature_indices].reshape(-1, len(numeric_feature_indices))

scaler = StandardScaler()
scaler.fit(all_numeric_inputs)

# 缩放输入
input_sequences[:, :, numeric_feature_indices] = scaler.transform(all_numeric_inputs).reshape(
    input_sequences.shape[0], input_sequences.shape[1], len(numeric_feature_indices))

# 缩放目标
all_numeric_targets = target_sequences.reshape(-1, len(numeric_feature_indices))
target_sequences = scaler.transform(all_numeric_targets).reshape(
    target_sequences.shape[0], target_sequences.shape[1], len(numeric_feature_indices))

# 准备基线模型的数据（不包含转弯特征）
input_features_baseline = ['center_x', 'center_y']

input_sequences_baseline = []
target_sequences_baseline = []
sequence_vehicle_ids_baseline = []

grouped = df_merged.groupby('id')

for track_id, group in grouped:
    group = group.sort_values('frame').reset_index(drop=True)
    features = group[input_features_baseline].values

    num_sequences = len(features) - sequence_length - predict_length + 1
    if num_sequences <= 0:
        continue

    for i in range(num_sequences):
        input_seq = features[i:i + sequence_length]
        target_seq = features[i + sequence_length:i + sequence_length + predict_length, :2]

        input_sequences_baseline.append(input_seq)
        target_sequences_baseline.append(target_seq)
        sequence_vehicle_ids_baseline.append(track_id)

# 转换为 NumPy 数组
input_sequences_baseline = np.array(input_sequences_baseline)
target_sequences_baseline = np.array(target_sequences_baseline)
sequence_vehicle_ids_baseline = np.array(sequence_vehicle_ids_baseline)

# 基线模型的数据标准化
all_numeric_inputs_baseline = input_sequences_baseline.reshape(-1, len(numeric_feature_indices))

scaler_baseline = StandardScaler()
scaler_baseline.fit(all_numeric_inputs_baseline)

# 缩放输入
input_sequences_baseline = scaler_baseline.transform(all_numeric_inputs_baseline).reshape(
    input_sequences_baseline.shape[0], input_sequences_baseline.shape[1], len(numeric_feature_indices))

# 缩放目标
all_numeric_targets_baseline = target_sequences_baseline.reshape(-1, len(numeric_feature_indices))
target_sequences_baseline = scaler_baseline.transform(all_numeric_targets_baseline).reshape(
    target_sequences_baseline.shape[0], target_sequences_baseline.shape[1], len(numeric_feature_indices))

# 定义模型
class TrajectoryPredictor(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=2, output_size=2):
        super(TrajectoryPredictor, self).__init__()
        self.lstm_encoder = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.lstm_decoder = nn.LSTM(output_size, hidden_size, num_layers, batch_first=True)
        self.fc_out = nn.Linear(hidden_size, output_size)

    def forward(self, x, target_len):
        batch_size = x.size(0)

        # 编码器
        _, (hidden, cell) = self.lstm_encoder(x)

        # 解码器
        decoder_input = x[:, -1, :2].unsqueeze(1)  # 从输入序列的最后一个位置开始
        outputs = []

        for t in range(target_len):
            out, (hidden, cell) = self.lstm_decoder(decoder_input, (hidden, cell))
            out = self.fc_out(out)
            outputs.append(out.squeeze(1))
            decoder_input = out  # 使用当前输出作为下一个输入

        outputs = torch.stack(outputs, dim=1)
        return outputs

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# 训练参数
batch_size = 1024
num_epochs = 10
target_len = predict_length  # 预测长度

# 准备当前模型的数据
input_size = input_sequences.shape[2]  # 输入特征数量
inputs = torch.tensor(input_sequences, dtype=torch.float32).to(device)
targets = torch.tensor(target_sequences, dtype=torch.float32).to(device)
model = TrajectoryPredictor(input_size=input_size, hidden_size=128, num_layers=2, output_size=2).to(device)

dataset = TensorDataset(inputs, targets)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

print(f'Total number of samples: {len(dataset)}')
print(f'Number of batches per epoch: {len(data_loader)}')

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 当前模型的训练循环
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch+1}/{num_epochs} for current model')
    model.train()
    total_loss = 0
    for batch_inputs, batch_targets in data_loader:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)
        optimizer.zero_grad()

        # 前向传播
        outputs = model(batch_inputs, target_len)

        # 计算损失
        loss = criterion(outputs, batch_targets)

        # 反向传播和优化
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(data_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}] for current model, Average Loss: {average_loss:.4f}')

# 准备基线模型的数据
input_size_baseline = input_sequences_baseline.shape[2]  # 输入特征数量（不包含转弯特征）
inputs_baseline = torch.tensor(input_sequences_baseline, dtype=torch.float32).to(device)
targets_baseline = torch.tensor(target_sequences_baseline, dtype=torch.float32).to(device)
model_baseline = TrajectoryPredictor(input_size=input_size_baseline, hidden_size=128, num_layers=2, output_size=2).to(device)

dataset_baseline = TensorDataset(inputs_baseline, targets_baseline)
data_loader_baseline = DataLoader(dataset_baseline, batch_size=batch_size, shuffle=True)

print(f'Total number of samples for baseline: {len(dataset_baseline)}')
print(f'Number of batches per epoch for baseline: {len(data_loader_baseline)}')

# 定义基线模型的损失函数和优化器
criterion_baseline = nn.MSELoss()
optimizer_baseline = torch.optim.Adam(model_baseline.parameters(), lr=0.001)

# 基线模型的训练循环
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch+1}/{num_epochs} for baseline model')
    model_baseline.train()
    total_loss = 0
    for batch_inputs, batch_targets in data_loader_baseline:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)
        optimizer_baseline.zero_grad()

        # 前向传播
        outputs = model_baseline(batch_inputs, target_len)

        # 计算损失
        loss = criterion_baseline(outputs, batch_targets)

        # 反向传播和优化
        loss.backward()
        optimizer_baseline.step()

        total_loss += loss.item()

    average_loss = total_loss / len(data_loader_baseline)
    print(f'Epoch [{epoch+1}/{num_epochs}] for baseline model, Average Loss: {average_loss:.4f}')

# 定义函数来计算指标
def compute_metrics(predictions, targets, horizons):
    metrics = {}
    for horizon in horizons:
        outputs_at_horizon = predictions[:, :horizon, :]  # [num_samples, horizon, 2]
        targets_at_horizon = targets[:, :horizon, :]

        # 计算误差
        errors = outputs_at_horizon - targets_at_horizon  # [num_samples, horizon, 2]
        squared_errors = errors ** 2
        mse = squared_errors.mean().item()
        rmse = np.sqrt(mse)

        abs_errors = errors.abs()
        mae = abs_errors.mean().item()

        # 计算 ADE
        displacement_errors = torch.norm(errors, dim=2)  # 计算 x 和 y 上的欧氏距离
        ade = displacement_errors.mean().item()

        # 计算 FDE
        final_errors = errors[:, -1, :]  # [num_samples, 2]
        fde = torch.norm(final_errors, dim=1).mean().item()

        metrics[horizon] = {
            'RMSE': rmse,
            'MAE': mae,
            'ADE': ade,
            'FDE': fde
        }

    return metrics

# 定义预测地平线
horizons = {
    15: 0.5,  # 0.5 秒（15 帧）
    30: 1.0,  # 1.0 秒（30 帧）
    45: 1.5   # 1.5 秒（45 帧）
}

# 过滤不超过 predict_length 的地平线
horizons = {k: v for k, v in horizons.items() if k <= predict_length}

# -----------------------------
# 模型在转弯车辆上的预测和比较
# -----------------------------

# 1. 筛选转弯车辆的序列
turn_labels = ['left_turn', 'right_turn']
turning_df = df_merged[df_merged['overall_turn_label'].isin(turn_labels)]

# 2. 准备转弯车辆的输入和目标序列
input_sequences_turn = []
target_sequences_turn = []
sequence_indices_turn = []

grouped_turning = turning_df.groupby('id')

for track_id, group in grouped_turning:
    group = group.sort_values('frame').reset_index(drop=True)
    features_current = group[input_features].values  # 包含转弯特征
    features_baseline = group[['center_x', 'center_y']].values  # 不包含转弯特征

    num_sequences = len(features_current) - sequence_length - predict_length + 1
    if num_sequences <= 0:
        continue

    for i in range(num_sequences):
        input_seq_current = features_current[i:i + sequence_length]
        input_seq_baseline = features_baseline[i:i + sequence_length]
        target_seq = features_current[i + sequence_length:i + sequence_length + predict_length, :2]  # 只取 'center_x' 和 'center_y'

        input_sequences_turn.append((input_seq_current, input_seq_baseline))
        target_sequences_turn.append(target_seq)
        sequence_indices_turn.append((track_id, i))

# 转换为 NumPy 数组
input_sequences_turn_current = np.array([seq[0] for seq in input_sequences_turn])
input_sequences_turn_baseline = np.array([seq[1] for seq in input_sequences_turn])
target_sequences_turn = np.array(target_sequences_turn)

# 对当前模型的数据进行标准化
# 输入
all_numeric_inputs_turn_current = input_sequences_turn_current[:, :, numeric_feature_indices].reshape(-1, len(numeric_feature_indices))
input_sequences_turn_current[:, :, numeric_feature_indices] = scaler.transform(all_numeric_inputs_turn_current).reshape(
    input_sequences_turn_current.shape[0], input_sequences_turn_current.shape[1], len(numeric_feature_indices))

# 目标
all_numeric_targets_turn = target_sequences_turn.reshape(-1, len(numeric_feature_indices))
target_sequences_turn_scaled = scaler.transform(all_numeric_targets_turn).reshape(
    target_sequences_turn.shape[0], target_sequences_turn.shape[1], len(numeric_feature_indices))

# 对基线模型的数据进行标准化
# 输入
all_numeric_inputs_turn_baseline = input_sequences_turn_baseline.reshape(-1, len(numeric_feature_indices))
input_sequences_turn_baseline = scaler_baseline.transform(all_numeric_inputs_turn_baseline).reshape(
    input_sequences_turn_baseline.shape[0], input_sequences_turn_baseline.shape[1], len(numeric_feature_indices))

# 目标
# 目标序列对于两个模型是相同的，但要使用各自的 scaler 进行标准化
target_sequences_turn_scaled_baseline = scaler_baseline.transform(all_numeric_targets_turn).reshape(
    target_sequences_turn.shape[0], target_sequences_turn.shape[1], len(numeric_feature_indices))

# 转换为张量
inputs_turn_current = torch.tensor(input_sequences_turn_current, dtype=torch.float32).to(device)
inputs_turn_baseline = torch.tensor(input_sequences_turn_baseline, dtype=torch.float32).to(device)
targets_turn_current = torch.tensor(target_sequences_turn_scaled, dtype=torch.float32).to(device)
targets_turn_baseline = torch.tensor(target_sequences_turn_scaled_baseline, dtype=torch.float32).to(device)

# 定义数据集和数据加载器
dataset_turn_current = TensorDataset(inputs_turn_current, targets_turn_current)
dataset_turn_baseline = TensorDataset(inputs_turn_baseline, targets_turn_baseline)

data_loader_turn_current = DataLoader(dataset_turn_current, batch_size=batch_size, shuffle=False)
data_loader_turn_baseline = DataLoader(dataset_turn_baseline, batch_size=batch_size, shuffle=False)

# 获取模型预测
def get_predictions(model, data_loader, target_len):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch_inputs, _ in data_loader:
            batch_inputs = batch_inputs.to(device)
            outputs = model(batch_inputs, target_len)
            predictions.append(outputs.cpu())
    predictions = torch.cat(predictions, dim=0)
    return predictions

# 获取当前模型的预测
predictions_turn_current = get_predictions(model, data_loader_turn_current, target_len)

# 获取基线模型的预测
predictions_turn_baseline = get_predictions(model_baseline, data_loader_turn_baseline, target_len)

# 反标准化预测结果
# 反标准化当前模型的预测
predictions_turn_current_unscaled = scaler.inverse_transform(predictions_turn_current.reshape(-1, 2)).reshape(
    predictions_turn_current.shape[0], predictions_turn_current.shape[1], 2)

# 反标准化基线模型的预测
predictions_turn_baseline_unscaled = scaler_baseline.inverse_transform(predictions_turn_baseline.reshape(-1, 2)).reshape(
    predictions_turn_baseline.shape[0], predictions_turn_baseline.shape[1], 2)

# 反标准化目标
targets_turn_unscaled = scaler.inverse_transform(targets_turn_current.reshape(-1, 2)).reshape(
    targets_turn_current.shape[0], targets_turn_current.shape[1], 2)

# 历史输入（用于可视化）
history_inputs_unscaled = scaler.inverse_transform(
    inputs_turn_current[:, :, numeric_feature_indices].reshape(-1, len(numeric_feature_indices))
).reshape(inputs_turn_current.shape[0], inputs_turn_current.shape[1], len(numeric_feature_indices))

# 定义函数来计算轨迹的曲率
def compute_trajectory_curvature(traj):
    # traj: [seq_len, 2]
    dx = np.gradient(traj[:, 0])
    dy = np.gradient(traj[:, 1])
    ddx = np.gradient(dx)
    ddy = np.gradient(dy)
    curvature = np.abs(dx * ddy - dy * ddx) / (dx * dx + dy * dy) ** 1.5
    curvature[np.isnan(curvature)] = 0
    curvature[np.isinf(curvature)] = 0
    return curvature

# 选择案例
num_samples = 5  # 可视化的样本数量
selected_indices = []

for idx in range(predictions_turn_current_unscaled.shape[0]):
    # 计算基线模型和当前模型预测的曲率
    curvature_baseline = compute_trajectory_curvature(predictions_turn_baseline_unscaled[idx])
    curvature_current = compute_trajectory_curvature(predictions_turn_current_unscaled[idx])

    # 计算曲率的平均值
    avg_curvature_baseline = np.mean(curvature_baseline)
    avg_curvature_current = np.mean(curvature_current)

    # 如果基线模型的曲率较小（预测为直行），而当前模型的曲率较大（预测为转弯），则选择该样本
    if avg_curvature_baseline < 0.001 and avg_curvature_current > avg_curvature_baseline + 0.005:
        selected_indices.append(idx)

    if len(selected_indices) >= num_samples:
        break

# 可视化选定的案例
for idx in selected_indices:
    plt.figure(figsize=(8, 6))

    # 绘制历史轨迹
    plt.plot(history_inputs_unscaled[idx, :, 0], history_inputs_unscaled[idx, :, 1], 'bo-', label='历史轨迹')

    # 绘制真实未来轨迹
    plt.plot(targets_turn_unscaled[idx, :, 0], targets_turn_unscaled[idx, :, 1], 'go-', label='真实未来轨迹')

    # 绘制基线模型预测
    plt.plot(predictions_turn_baseline_unscaled[idx, :, 0], predictions_turn_baseline_unscaled[idx, :, 1], 'ro--', label='基线模型预测')

    # 绘制当前模型预测
    plt.plot(predictions_turn_current_unscaled[idx, :, 0], predictions_turn_current_unscaled[idx, :, 1], 'co--', label='当前模型预测')

    plt.legend()
    plt.xlabel('center_x')
    plt.ylabel('center_y')
    vehicle_id, seq_idx = sequence_indices_turn[idx]
    plt.title(f'车辆ID {vehicle_id} 序列 {seq_idx} 的轨迹预测比较')

    # 保存或显示图像
    # plt.show()
    # 或者保存图像
    figure_path = output_dir / f'Comparison_vehicle_{vehicle_id}_seq_{seq_idx}_{timestamp}.png'
    plt.savefig(figure_path)
    plt.close()
    print(f'Plot saved to {figure_path}')


Outputs will be saved to: eval_model_on_video/output_20241018_082033
Number of missing 'overall_turn_label': 0
Using device: cuda
Total number of samples: 239560
Number of batches per epoch: 234
Starting epoch 1/10 for current model
Epoch [1/10] for current model, Average Loss: 0.1221
Starting epoch 2/10 for current model
Epoch [2/10] for current model, Average Loss: 0.0044
Starting epoch 3/10 for current model
Epoch [3/10] for current model, Average Loss: 0.0032
Starting epoch 4/10 for current model
Epoch [4/10] for current model, Average Loss: 0.0026
Starting epoch 5/10 for current model
Epoch [5/10] for current model, Average Loss: 0.0022
Starting epoch 6/10 for current model
Epoch [6/10] for current model, Average Loss: 0.0016
Starting epoch 7/10 for current model
Epoch [7/10] for current model, Average Loss: 0.0403
Starting epoch 8/10 for current model
Epoch [8/10] for current model, Average Loss: 0.0051
Starting epoch 9/10 for current model
Epoch [9/10] for current model, Average

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [8]:
# 导入必要的库
import os
import datetime
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import cv2
import joblib
import warnings
import random
warnings.filterwarnings('ignore')

# 定义路径
path = Path('csv_out')
eval_video_path = Path('eval_model_on_video')

# 创建带有时间戳的新输出目录，避免覆盖
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
output_dir = eval_video_path / f'output_{timestamp}'
output_dir.mkdir(parents=True, exist_ok=True)
print(f'Outputs will be saved to: {output_dir}')

# 加载数据
df1 = pd.read_csv(path / 'tracking_data.csv')
df2 = pd.read_csv(path / 'overall_turn_label.csv')

# 合并数据帧
df_merged = pd.merge(df1, df2[['id', 'frame', 'overall_turn_label']], on=['id', 'frame'], how='left')

# 按车辆ID填充缺失的 'overall_turn_label' 值（前向和后向填充）
df_merged['overall_turn_label'] = df_merged.groupby('id')['overall_turn_label'].fillna(method='ffill')
df_merged['overall_turn_label'] = df_merged.groupby('id')['overall_turn_label'].fillna(method='bfill')

# 检查剩余的缺失值
missing_values = df_merged['overall_turn_label'].isnull().sum()
print(f"Number of missing 'overall_turn_label': {missing_values}")

# 填充任何剩余的缺失值为 'straight'（如果有）
df_merged['overall_turn_label'] = df_merged['overall_turn_label'].fillna('straight')

# One-Hot 编码 'overall_turn_label'
encoder = OneHotEncoder(sparse_output=False)
turn_labels_encoded = encoder.fit_transform(df_merged[['overall_turn_label']])
turn_label_columns = encoder.get_feature_names_out(['overall_turn_label'])
df_merged[turn_label_columns] = turn_labels_encoded

# 定义输入特征
input_features = ['center_x', 'center_y'] + list(turn_label_columns)

# 定义序列长度
sequence_length = 180  # 输入序列长度（180帧，30fps下为6秒）
predict_length = 90    # 最大预测序列长度（90帧，30fps下为3秒）

# 定义预测地平线
horizons = {
    30: 1.0,  # 1.0 秒（30 帧）
    60: 2.0,  # 2.0 秒（60 帧）
    90: 3.0   # 3.0 秒（90 帧）
}

# 生成当前模型的输入和目标序列（包含转弯特征）
input_sequences = []
target_sequences = []
sequence_vehicle_ids = []

grouped = df_merged.groupby('id')

for track_id, group in grouped:
    group = group.sort_values('frame').reset_index(drop=True)
    features = group[input_features].values

    num_sequences = len(features) - sequence_length - predict_length + 1
    if num_sequences <= 0:
        continue

    for i in range(num_sequences):
        input_seq = features[i:i + sequence_length]
        target_seq = features[i + sequence_length:i + sequence_length + predict_length, :2]  # 只取 'center_x' 和 'center_y'

        input_sequences.append(input_seq)
        target_sequences.append(target_seq)
        sequence_vehicle_ids.append(track_id)

# 转换为 NumPy 数组
input_sequences = np.array(input_sequences)
target_sequences = np.array(target_sequences)
sequence_vehicle_ids = np.array(sequence_vehicle_ids)

# 数据标准化
numeric_feature_indices = [0, 1]  # 'center_x' 和 'center_y' 的索引

# 扁平化输入以进行缩放
all_numeric_inputs = input_sequences[:, :, numeric_feature_indices].reshape(-1, len(numeric_feature_indices))

scaler = StandardScaler()
scaler.fit(all_numeric_inputs)

# 缩放输入
input_sequences[:, :, numeric_feature_indices] = scaler.transform(all_numeric_inputs).reshape(
    input_sequences.shape[0], input_sequences.shape[1], len(numeric_feature_indices))

# 缩放目标
all_numeric_targets = target_sequences.reshape(-1, len(numeric_feature_indices))
target_sequences = scaler.transform(all_numeric_targets).reshape(
    target_sequences.shape[0], target_sequences.shape[1], len(numeric_feature_indices))

# 准备基线模型的数据（不包含转弯特征）
input_features_baseline = ['center_x', 'center_y']

input_sequences_baseline = []
target_sequences_baseline = []
sequence_vehicle_ids_baseline = []

grouped = df_merged.groupby('id')

for track_id, group in grouped:
    group = group.sort_values('frame').reset_index(drop=True)
    features = group[input_features_baseline].values

    num_sequences = len(features) - sequence_length - predict_length + 1
    if num_sequences <= 0:
        continue

    for i in range(num_sequences):
        input_seq = features[i:i + sequence_length]
        target_seq = features[i + sequence_length:i + sequence_length + predict_length, :2]

        input_sequences_baseline.append(input_seq)
        target_sequences_baseline.append(target_seq)
        sequence_vehicle_ids_baseline.append(track_id)

# 转换为 NumPy 数组
input_sequences_baseline = np.array(input_sequences_baseline)
target_sequences_baseline = np.array(target_sequences_baseline)
sequence_vehicle_ids_baseline = np.array(sequence_vehicle_ids_baseline)

# 基线模型的数据标准化
all_numeric_inputs_baseline = input_sequences_baseline.reshape(-1, len(numeric_feature_indices))

scaler_baseline = StandardScaler()
scaler_baseline.fit(all_numeric_inputs_baseline)

# 缩放输入
input_sequences_baseline = scaler_baseline.transform(all_numeric_inputs_baseline).reshape(
    input_sequences_baseline.shape[0], input_sequences_baseline.shape[1], len(numeric_feature_indices))

# 缩放目标
all_numeric_targets_baseline = target_sequences_baseline.reshape(-1, len(numeric_feature_indices))
target_sequences_baseline = scaler_baseline.transform(all_numeric_targets_baseline).reshape(
    target_sequences_baseline.shape[0], target_sequences_baseline.shape[1], len(numeric_feature_indices))

# 定义模型
class TrajectoryPredictor(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=2, output_size=2):
        super(TrajectoryPredictor, self).__init__()
        self.lstm_encoder = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.lstm_decoder = nn.LSTM(output_size, hidden_size, num_layers, batch_first=True)
        self.fc_out = nn.Linear(hidden_size, output_size)

    def forward(self, x, target_len):
        batch_size = x.size(0)

        # 编码器
        _, (hidden, cell) = self.lstm_encoder(x)

        # 解码器
        decoder_input = x[:, -1, :2].unsqueeze(1)  # 从输入序列的最后一个位置开始
        outputs = []

        for t in range(target_len):
            out, (hidden, cell) = self.lstm_decoder(decoder_input, (hidden, cell))
            out = self.fc_out(out)
            outputs.append(out.squeeze(1))
            decoder_input = out  # 使用当前输出作为下一个输入

        outputs = torch.stack(outputs, dim=1)
        return outputs

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# 训练参数
batch_size = 512  # 根据内存情况调整批次大小
num_epochs = 10
target_len = predict_length  # 最大预测长度

# 准备当前模型的数据
input_size = input_sequences.shape[2]  # 输入特征数量
inputs = torch.tensor(input_sequences, dtype=torch.float32).to(device)
targets = torch.tensor(target_sequences, dtype=torch.float32).to(device)
model = TrajectoryPredictor(input_size=input_size, hidden_size=128, num_layers=2, output_size=2).to(device)

dataset = TensorDataset(inputs, targets)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

print(f'Total number of samples: {len(dataset)}')
print(f'Number of batches per epoch: {len(data_loader)}')

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 当前模型的训练循环
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch+1}/{num_epochs} for current model')
    model.train()
    total_loss = 0
    for batch_inputs, batch_targets in data_loader:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)
        optimizer.zero_grad()

        # 前向传播
        outputs = model(batch_inputs, target_len)

        # 计算损失
        loss = criterion(outputs, batch_targets)

        # 反向传播和优化
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(data_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}] for current model, Average Loss: {average_loss:.4f}')

# 准备基线模型的数据
input_size_baseline = input_sequences_baseline.shape[2]  # 输入特征数量（不包含转弯特征）
inputs_baseline = torch.tensor(input_sequences_baseline, dtype=torch.float32).to(device)
targets_baseline = torch.tensor(target_sequences_baseline, dtype=torch.float32).to(device)
model_baseline = TrajectoryPredictor(input_size=input_size_baseline, hidden_size=128, num_layers=2, output_size=2).to(device)

dataset_baseline = TensorDataset(inputs_baseline, targets_baseline)
data_loader_baseline = DataLoader(dataset_baseline, batch_size=batch_size, shuffle=True)

print(f'Total number of samples for baseline: {len(dataset_baseline)}')
print(f'Number of batches per epoch for baseline: {len(data_loader_baseline)}')

# 定义基线模型的损失函数和优化器
criterion_baseline = nn.MSELoss()
optimizer_baseline = torch.optim.Adam(model_baseline.parameters(), lr=0.001)

# 基线模型的训练循环
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch+1}/{num_epochs} for baseline model')
    model_baseline.train()
    total_loss = 0
    for batch_inputs, batch_targets in data_loader_baseline:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)
        optimizer_baseline.zero_grad()

        # 前向传播
        outputs = model_baseline(batch_inputs, target_len)

        # 计算损失
        loss = criterion_baseline(outputs, batch_targets)

        # 反向传播和优化
        loss.backward()
        optimizer_baseline.step()

        total_loss += loss.item()

    average_loss = total_loss / len(data_loader_baseline)
    print(f'Epoch [{epoch+1}/{num_epochs}] for baseline model, Average Loss: {average_loss:.4f}')

# 定义函数来计算指标
def compute_metrics(predictions, targets, horizons):
    metrics = {}
    for horizon in horizons:
        outputs_at_horizon = predictions[:, :horizon, :]  # [num_samples, horizon, 2]
        targets_at_horizon = targets[:, :horizon, :]

        # 计算误差
        errors = outputs_at_horizon - targets_at_horizon  # [num_samples, horizon, 2]
        squared_errors = errors ** 2
        mse = squared_errors.mean().item()
        rmse = np.sqrt(mse)

        abs_errors = errors.abs()
        mae = abs_errors.mean().item()

        # 计算 ADE
        displacement_errors = torch.norm(errors, dim=2)  # 计算 x 和 y 上的欧氏距离
        ade = displacement_errors.mean().item()

        # 计算 FDE
        final_errors = errors[:, -1, :]  # [num_samples, 2]
        fde = torch.norm(final_errors, dim=1).mean().item()

        metrics[horizon] = {
            'RMSE': rmse,
            'MAE': mae,
            'ADE': ade,
            'FDE': fde
        }

    return metrics

# 评估当前模型
model.eval()
with torch.no_grad():
    total_outputs = []
    total_targets = []
    for batch_inputs, batch_targets in data_loader:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)

        outputs = model(batch_inputs, target_len)
        total_outputs.append(outputs.cpu())
        total_targets.append(batch_targets.cpu())

    total_outputs = torch.cat(total_outputs, dim=0)
    total_targets = torch.cat(total_targets, dim=0)

    metrics = compute_metrics(total_outputs, total_targets, horizons.keys())

    # 保存指标到文件
    metrics_file = output_dir / 'metrics_current_model.txt'
    with open(metrics_file, 'w') as f:
        for horizon_frames, time_sec in horizons.items():
            print(f'\nMetrics for current model at horizon: {time_sec} seconds ({horizon_frames} frames)')
            print(f"RMSE: {metrics[horizon_frames]['RMSE']:.4f}")
            print(f"MAE: {metrics[horizon_frames]['MAE']:.4f}")
            print(f"ADE: {metrics[horizon_frames]['ADE']:.4f}")
            print(f"FDE: {metrics[horizon_frames]['FDE']:.4f}")

            f.write(f'Metrics for current model at horizon: {time_sec} seconds ({horizon_frames} frames)\n')
            f.write(f"RMSE: {metrics[horizon_frames]['RMSE']:.4f}\n")
            f.write(f"MAE: {metrics[horizon_frames]['MAE']:.4f}\n")
            f.write(f"ADE: {metrics[horizon_frames]['ADE']:.4f}\n")
            f.write(f"FDE: {metrics[horizon_frames]['FDE']:.4f}\n\n")

# 评估基线模型
model_baseline.eval()
with torch.no_grad():
    total_outputs_baseline = []
    total_targets_baseline = []
    for batch_inputs, batch_targets in data_loader_baseline:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)

        outputs = model_baseline(batch_inputs, target_len)
        total_outputs_baseline.append(outputs.cpu())
        total_targets_baseline.append(batch_targets.cpu())

    total_outputs_baseline = torch.cat(total_outputs_baseline, dim=0)
    total_targets_baseline = torch.cat(total_targets_baseline, dim=0)

    metrics_baseline = compute_metrics(total_outputs_baseline, total_targets_baseline, horizons.keys())

    # 保存指标到文件
    metrics_file_baseline = output_dir / 'metrics_baseline_model.txt'
    with open(metrics_file_baseline, 'w') as f:
        for horizon_frames, time_sec in horizons.items():
            print(f'\nMetrics for baseline model at horizon: {time_sec} seconds ({horizon_frames} frames)')
            print(f"RMSE: {metrics_baseline[horizon_frames]['RMSE']:.4f}")
            print(f"MAE: {metrics_baseline[horizon_frames]['MAE']:.4f}")
            print(f"ADE: {metrics_baseline[horizon_frames]['ADE']:.4f}")
            print(f"FDE: {metrics_baseline[horizon_frames]['FDE']:.4f}")

            f.write(f"Metrics for baseline model at horizon: {time_sec} seconds ({horizon_frames} frames)\n")
            f.write(f"RMSE: {metrics_baseline[horizon_frames]['RMSE']:.4f}\n")
            f.write(f"MAE: {metrics_baseline[horizon_frames]['MAE']:.4f}\n")
            f.write(f"ADE: {metrics_baseline[horizon_frames]['ADE']:.4f}\n")
            f.write(f"FDE: {metrics_baseline[horizon_frames]['FDE']:.4f}\n\n")

# 保存模型和标准化器
model_file = output_dir / f'trajectory_predictor_current_{timestamp}.pth'
scaler_file = output_dir / f'scaler_current_{timestamp}.save'

torch.save(model.state_dict(), model_file)
joblib.dump(scaler, scaler_file)
print(f'Current model saved to {model_file}')
print(f'Current scaler saved to {scaler_file}')

model_file_baseline = output_dir / f'trajectory_predictor_baseline_{timestamp}.pth'
scaler_file_baseline = output_dir / f'scaler_baseline_{timestamp}.save'

torch.save(model_baseline.state_dict(), model_file_baseline)
joblib.dump(scaler_baseline, scaler_file_baseline)
print(f'Baseline model saved to {model_file_baseline}')
print(f'Baseline scaler saved to {scaler_file_baseline}')

# 后续的可视化和在转弯车辆上的评估代码保持不变
# ...

# ---------------------------------
# 在转弯车辆上评估模型
# ---------------------------------

# 1. 筛选转弯车辆的数据
turn_labels = ['left_turn', 'right_turn']
turning_df = df_merged[df_merged['overall_turn_label'].isin(turn_labels)]

# 2. 准备转弯车辆的输入和目标序列
input_sequences_turn = []
target_sequences_turn = []

grouped_turning = turning_df.groupby('id')

for track_id, group in grouped_turning:
    group = group.sort_values('frame').reset_index(drop=True)
    features = group[input_features].values

    num_sequences = len(features) - sequence_length - predict_length + 1
    if num_sequences <= 0:
        continue

    for i in range(num_sequences):
        input_seq = features[i:i + sequence_length]
        target_seq = features[i + sequence_length:i + sequence_length + predict_length, :2]  # 只取 'center_x' 和 'center_y'

        input_sequences_turn.append(input_seq)
        target_sequences_turn.append(target_seq)

# 转换为 NumPy 数组
input_sequences_turn = np.array(input_sequences_turn)
target_sequences_turn = np.array(target_sequences_turn)

# 数据标准化（使用相同的 scaler）
numeric_feature_indices = [0, 1]  # 'center_x', 'center_y'

# 缩放输入
all_numeric_inputs_turn = input_sequences_turn[:, :, numeric_feature_indices].reshape(-1, len(numeric_feature_indices))
input_sequences_turn[:, :, numeric_feature_indices] = scaler.transform(all_numeric_inputs_turn).reshape(
    input_sequences_turn.shape[0], input_sequences_turn.shape[1], len(numeric_feature_indices))

# 缩放目标
all_numeric_targets_turn = target_sequences_turn.reshape(-1, len(numeric_feature_indices))
target_sequences_turn = scaler.transform(all_numeric_targets_turn).reshape(
    target_sequences_turn.shape[0], target_sequences_turn.shape[1], len(numeric_feature_indices))

# 创建数据集和数据加载器
inputs_turn = torch.tensor(input_sequences_turn, dtype=torch.float32).to(device)
targets_turn = torch.tensor(target_sequences_turn, dtype=torch.float32).to(device)

dataset_turn = TensorDataset(inputs_turn, targets_turn)
data_loader_turn = DataLoader(dataset_turn, batch_size=batch_size, shuffle=False)

# 3. 在转弯车辆上评估当前模型
def evaluate_model_on_turning(model, data_loader, target_len):
    model.eval()
    with torch.no_grad():
        total_outputs = []
        total_targets = []
        for batch_inputs, batch_targets in data_loader:
            batch_inputs = batch_inputs.to(device)
            batch_targets = batch_targets.to(device)

            outputs = model(batch_inputs, target_len)
            total_outputs.append(outputs.cpu())
            total_targets.append(batch_targets.cpu())

        total_outputs = torch.cat(total_outputs, dim=0)
        total_targets = torch.cat(total_targets, dim=0)

        metrics = compute_metrics(total_outputs, total_targets, horizons.keys())
    return metrics

metrics_turn_current = evaluate_model_on_turning(model, data_loader_turn, target_len)

# 4. 为基线模型准备转弯车辆的数据
# 移除转弯特征
input_sequences_turn_baseline = input_sequences_turn[:, :, :2]  # 只保留 'center_x' 和 'center_y'

# 使用基线模型的 scaler 缩放输入
all_numeric_inputs_turn_baseline = input_sequences_turn_baseline.reshape(-1, len(numeric_feature_indices))
input_sequences_turn_baseline = scaler_baseline.transform(all_numeric_inputs_turn_baseline).reshape(
    input_sequences_turn_baseline.shape[0], input_sequences_turn_baseline.shape[1], len(numeric_feature_indices))

# 使用基线模型的 scaler 缩放目标
all_numeric_targets_turn_baseline = target_sequences_turn.reshape(-1, len(numeric_feature_indices))
target_sequences_turn_baseline = scaler_baseline.transform(all_numeric_targets_turn_baseline).reshape(
    target_sequences_turn_baseline.shape[0], target_sequences_turn_baseline.shape[1], len(numeric_feature_indices))

# 创建数据集和数据加载器
inputs_turn_baseline = torch.tensor(input_sequences_turn_baseline, dtype=torch.float32).to(device)
targets_turn_baseline = torch.tensor(target_sequences_turn_baseline, dtype=torch.float32).to(device)

dataset_turn_baseline = TensorDataset(inputs_turn_baseline, targets_turn_baseline)
data_loader_turn_baseline = DataLoader(dataset_turn_baseline, batch_size=batch_size, shuffle=False)

# 在转弯车辆上评估基线模型
metrics_turn_baseline = evaluate_model_on_turning(model_baseline, data_loader_turn_baseline, target_len)

# 5. 比较结果
metrics_file_turn = output_dir / 'metrics_turning_vehicles.txt'
with open(metrics_file_turn, 'w') as f:
    for horizon_frames, time_sec in horizons.items():
        print(f'\nMetrics on turning vehicles at horizon: {time_sec} seconds ({horizon_frames} frames)')
        print(f"Baseline Model:")
        print(f"  RMSE: {metrics_turn_baseline[horizon_frames]['RMSE']:.4f}")
        print(f"  MAE: {metrics_turn_baseline[horizon_frames]['MAE']:.4f}")
        print(f"  ADE: {metrics_turn_baseline[horizon_frames]['ADE']:.4f}")
        print(f"  FDE: {metrics_turn_baseline[horizon_frames]['FDE']:.4f}")
        print(f"Current Model:")
        print(f"  RMSE: {metrics_turn_current[horizon_frames]['RMSE']:.4f}")
        print(f"  MAE: {metrics_turn_current[horizon_frames]['MAE']:.4f}")
        print(f"  ADE: {metrics_turn_current[horizon_frames]['ADE']:.4f}")
        print(f"  FDE: {metrics_turn_current[horizon_frames]['FDE']:.4f}")

        f.write(f'Metrics on turning vehicles at horizon: {time_sec} seconds ({horizon_frames} frames)\n')
        f.write(f"Baseline Model:\n")
        f.write(f"  RMSE: {metrics_turn_baseline[horizon_frames]['RMSE']:.4f}\n")
        f.write(f"  MAE: {metrics_turn_baseline[horizon_frames]['MAE']:.4f}\n")
        f.write(f"  ADE: {metrics_turn_baseline[horizon_frames]['ADE']:.4f}\n")
        f.write(f"  FDE: {metrics_turn_baseline[horizon_frames]['FDE']:.4f}\n")
        f.write(f"Current Model:\n")
        f.write(f"  RMSE: {metrics_turn_current[horizon_frames]['RMSE']:.4f}\n")
        f.write(f"  MAE: {metrics_turn_current[horizon_frames]['MAE']:.4f}\n")
        f.write(f"  ADE: {metrics_turn_current[horizon_frames]['ADE']:.4f}\n")
        f.write(f"  FDE: {metrics_turn_current[horizon_frames]['FDE']:.4f}\n\n")

print(f"Metrics on turning vehicles saved to {metrics_file_turn}")


Outputs will be saved to: eval_model_on_video/output_20241018_082709
Number of missing 'overall_turn_label': 0
Using device: cuda
Total number of samples: 212539
Number of batches per epoch: 416
Starting epoch 1/10 for current model
Epoch [1/10] for current model, Average Loss: 0.1269
Starting epoch 2/10 for current model
Epoch [2/10] for current model, Average Loss: 0.0067
Starting epoch 3/10 for current model
Epoch [3/10] for current model, Average Loss: 0.0046
Starting epoch 4/10 for current model
Epoch [4/10] for current model, Average Loss: 0.0043
Starting epoch 5/10 for current model
Epoch [5/10] for current model, Average Loss: 0.0218
Starting epoch 6/10 for current model
Epoch [6/10] for current model, Average Loss: 0.0047
Starting epoch 7/10 for current model
Epoch [7/10] for current model, Average Loss: 0.0033
Starting epoch 8/10 for current model
Epoch [8/10] for current model, Average Loss: 0.0028
Starting epoch 9/10 for current model
Epoch [9/10] for current model, Average

NameError: name 'target_sequences_turn_baseline' is not defined